# <center> DENİZ BALCI SİMÜLASYON 1.ÖDEVİ</center>

# 0)  Paket Kurulumu ve Ön açıklama

Bütün hücreleri ayrı ayrı veya bütün halde çalıştırabilirsiniz.İlk olarak paketleri yükleyelim.

In [1]:
!pip install scipy
!pip install numpy

Defaulting to user installation because normal site-packages is not writeable

[notice] A new release of pip is available: 23.3.1 -> 24.0
[notice] To update, run: python3 -m pip install --upgrade pip
Defaulting to user installation because normal site-packages is not writeable

[notice] A new release of pip is available: 23.3.1 -> 24.0
[notice] To update, run: python3 -m pip install --upgrade pip


## 1)Lineer kongrüans üreteçler

In [2]:
def linearkong(seed: float, a: int, b: int, m: int, n: int) -> list:
    generated_numbers = []
    for _ in range(n):
        x = (a * seed + b) % m
        print(x)
        seed = x
        generated_numbers.append(seed)
    return generated_numbers


print(linearkong(73, 5, 2, 123, 10))  

121
115
85
58
46
109
55
31
34
49
[121, 115, 85, 58, 46, 109, 55, 31, 34, 49]


## 2)DAĞILIM SİMÜLASYONLARI

Ödevimizde 5 kesikli 5 sürekli dağılımın simülasyon hazırlanması istenilmişti.

### 2.1)3 parametreli weibull dağılımı

$$ F(t,\alpha,\beta,\eta)= \left\{ \begin{array}{cl}
1-e^{-((t-\alpha)/ \eta)} &  t > \alpha \\
0 &  t \leq \alpha
\end{array} \right.$$



$$f(t,\alpha,\beta,\eta)= \left\{ \begin{array}{cl}
      \frac{\beta}{\eta}(\frac{t-\alpha}{\eta})^{\beta-1} e^{-((t-\alpha)/ \eta)}&  t > \alpha \\
0 &  t \leq \alpha
\end{array} \right.$$

$$ F(x)^{-1}=ln(1-x)+\alpha$$

$$\mu=\eta\Gamma(1+\frac{1}{\beta})+\alpha$$


$$\sigma^2=\eta^2[\Gamma(1+\frac{2}{\beta})-\Gamma^2(1+\frac{1}{\beta}) ]$$





In [3]:
import numpy as np
import scipy.stats
import math

def mean_of_threeweibull(alpha: float, beta: float, eta: float)->float:
    assert all(val >= 0 for val in [alpha, beta, eta])
    
    result = eta * (np.math.gamma(1 + (1 / beta))) + alpha
    return result

def variance_of_threeweibull(alpha: float, beta: float, eta: float)->float:
    assert all(val >= 0 for val in [alpha, beta, eta])
    
    result = (eta ** 2) * (np.math.gamma(1 + (2 / beta)) - (np.math.gamma(1 + (1 / beta))) ** 2)
    return result

def inverse_of_threeweibull(p: float, alpha: float, beta: float, eta: float)->float:
    assert all(val >= 0 for val in [alpha, beta, eta])
    
    quantile_value = alpha + eta * (-np.log(1 - p)) ** (1 / beta)
    return quantile_value


def threeweibulldatagenerator(n:int, alpha: float, beta: float, eta: float)->list:
    assert all(val >= 0 for val in [alpha, beta, eta]) and n > 0
    
    generated_data = np.random.uniform(0, 1, n)
    dataset = [inverse_of_threeweibull(i, alpha, beta, eta) for i in generated_data]
    return dataset

## FONKSİYON PARAMETRELERİ
n=1000
alpha=1
beta=2
eta=3
x=threeweibulldatagenerator(n,alpha,beta,eta)

## ÖRNEKLEM PARAMETRELERİ

m1 = np.mean(x)
m2 = np.var(x)
m3 = scipy.stats.skew(x)
m4 = scipy.stats.kurtosis(x)
## ANAKÜTLE PARAMETRELERİ
M1=mean_of_threeweibull(alpha, beta, eta)
M2=variance_of_threeweibull(alpha, beta, eta)



print("Örneklem parametreleri")
print("Ortalama :{:.4f} -Varyans:{:.4f} -çarpıklık: {:.4f} -basıklık :{:.4f}".format(m1,m2,m3,m4))
print("Anakütle  parametreleri")
print("Ortalama :{:.4f} -Varyans:{:.4f} ".format(M1, M2))

Örneklem parametreleri
Ortalama :3.6334 -Varyans:1.8337 -çarpıklık: 0.5439 -basıklık :-0.0987
Anakütle  parametreleri
Ortalama :3.6587 -Varyans:1.9314 


### 2.2)Laplace Dağılımı

$$ F(a,b)= \left\{ \begin{array}{cl}
1/2e^{-((a-x)/b) } &  x < a \\
1-1/2e^{-((a-x)/b) } &  x \ge a
\end{array} \right.$$

$$ F(a,b)^{-1}= \left\{ \begin{array}{cl}
2ln(2y)+a &  x < a \\
bln(2(1-y))+a &  x \ge a
\end{array} \right.$$


$$ f(x)=1/2b-e^{-((|a-x|)/b) }  \quad -\infty<x<\infty $$
$$ \mu =a$$
$$ \sigma^2 =2b^2$$
r inci ortalama etrafındaki moment
$$ \mu_r= \left\{ \begin{array}{cl}
r!b^r &  r \quad \text{çift} \\
0 &  r \quad \text{tek}
\end{array} \right.      $$

In [4]:
import numpy as np
import scipy.stats

def laplacecdf(a: float, b: float, x: float) -> float:
    if x < a:
        return (1 / (2)) * math.exp(-(a - x) / b)
    else:
        return 1 - (1 / (2)) * math.exp(-(a - x) / b)


def laplacepdf(a: float, b: float, x: float) -> float:
    return (1 / (2 * b)) * math.exp(-abs((a - x) / b))


def laplacemean(a: float) -> float:
    return a


def laplacevariance(b: float) -> float:
    return 2 * (b * b)


def laplacemoment(a: float, b: float, r: int) -> float:
    if r % 2 == 0:
        return math.factorial(r) * (math.pow(r, 2))
    else:
        return 0
    
def inverse_laplacecdf(x: float, a: float, b: float)-> float:
    if x <a:
        return 2 * np.log(2 * x) + a
    else:
        return b * np.log(2 * (1 - x)) + a
    
    
    
def laplacedatagenerator(n:int, a: float, b: float)->list:   
    generated_data = np.random.uniform(0, 1, n)
    dataset = [inverse_laplacecdf(i, a, b) for i in generated_data]
    return dataset




## FONKSİYON PARAMETRELERİ
n=10
a=1
b=2

x=laplacedatagenerator(n,a,b)

## ÖRNEKLEM PARAMETRELERİ

m1 = np.mean(x)
m2 = np.var(x)
m3 = scipy.stats.skew(x)
m4 = scipy.stats.kurtosis(x)
## ANAKÜTLE PARAMETRELERİ
M1=laplacemean(a)
M2=laplacevariance(b)



print("Örneklem parametreleri")
print("Ortalama :{:.4f} -Varyans:{:.4f} -çarpıklık: {:.4f} -basıklık :{:.4f}".format(m1,m2,m3,m4))
print("Anakütle  parametreleri")
print("Ortalama :{:.4f} -Varyans:{:.4f} ".format(M1, M2))


Örneklem parametreleri
Ortalama :-0.6667 -Varyans:5.4929 -çarpıklık: -0.3729 -basıklık :-1.1769
Anakütle  parametreleri
Ortalama :1.0000 -Varyans:8.0000 


### 2.3)Rayleigh Dağılım

$$ F(x)=1-exp(-x^2/(2b^2))$$
$$ f(x)= (x/b^2)exp(-x^2/(2b^2))$$
$$ F(x)^{-1}=\sqrt{-2b^2(ln(1-y))}$$
$$\E(X^n) = b^n 2^{n/2} \Gamma(1 + n/2)$$
$$  \mu=b(\pi/2)^{(1/2)}$$
$$  \sigma^2= (2-(\pi/2)^{(1/2)}$$
$$ \skw(X) = 2 \sqrt{\pi}(\pi - 3) \big/ (4 - \pi)^{3/2} \approx 0.6311$$

In [5]:
import numpy as np
import scipy.stats

def rayleigh_cdf(x:float, b:float)->float:
    return 1 - np.exp(-x**2 / (2 * b**2))

def rayleigh_pdf(x:float, b:float)->float:
    return (x / (b**2)) * np.exp(-x**2 / (2 * b**2))

def rayleigh_mean(b:float)->float:
    return b * np.sqrt(np.pi / 2)

def rayleigh_variance(b:float)->float:
    return 2 * (1 - np.pi / (4 * np.sqrt(2)))


def inverse_rayleigh(x:float, b:float)->float:
    return np.sqrt(-2 * b**2 * np.log(1 - x))



def rayleighdatagenerator(n:int, b:float)->list:   
    generated_data = np.random.uniform(0, 1, n)
    dataset = [inverse_rayleigh(i,b) for i in generated_data]
    return dataset

## FONKSİYON PARAMETRELERİ
n=10000
b = 1.0  

x=rayleighdatagenerator(n,b)
## ÖRNEKLEM PARAMETRELERİ

m1 = np.mean(x)
m2 = np.var(x)
m3 = scipy.stats.skew(x)
m4 = scipy.stats.kurtosis(x)

## ANAKÜTLE PARAMETRELERİ
M1= rayleigh_mean(b)
M2=rayleigh_variance(b)

print("Örneklem parametreleri")
print("Ortalama :{:.4f} -Varyans:{:.4f} -çarpıklık: {:.4f} -basıklık :{:.4f}".format(m1,m2,m3,m4))
print("Anakütle  parametreleri")
print("Ortalama :{:.4f} -Varyans:{:.4f} -çarpıklık: 0.6311 ".format(M1, M2,))



Örneklem parametreleri
Ortalama :1.2530 -Varyans:0.4283 -çarpıklık: 0.6251 -basıklık :0.1682
Anakütle  parametreleri
Ortalama :1.2533 -Varyans:0.8893 -çarpıklık: 0.6311 


### 2.4)General Hyperbolic Secant Distribution

$$ F(x) = \frac{2}{\pi} \arctan\left\{\exp\left[\frac{\pi}{2}\left(\frac{x - \mu}{\sigma}\right)\right]\right\}, \quad x \in \R $$
$$ F(x)^{-1} =2\sigma/ \pi ln(tan( \pi,y/2))+\mu, \quad x \in \R $$
$$ f(x) = \frac{1}{2 \sigma} \sech\left[\frac{\pi}{2}\left(\frac{x - \mu}{\sigma}\right)\right], \quad x \in \R$$
$$ M(t) = e^{\mu t} \sec(\sigma t), \quad t \in \left(-\frac{\pi}{2 \sigma}, \frac{\pi}{2 \sigma}\right) $$ 
$$ \E(X) = \mu $$
$$ \var(X) = \sigma^2$$
$$\skw(X) = 0 $$
$$\kur(X) = 5$$

In [6]:
import numpy as np
import scipy.stats



def f(x:float, mu:float, sigma:float)->float:
    return 1 / (2 * sigma) * 1 / np.cosh(np.pi / 2 * (x - mu) / sigma)

def F(x:float, mu:float, sigma:float)->float:
    arg = np.pi / 2 * (x - mu) / sigma
    return 2 / np.pi * np.arctan(np.exp(arg))

def hyperbolic_inverse(x:float, mu:float, sigma:float)->float:
    return 2 * sigma / np.pi * np.log(np.tan(np.pi * x / 2)) + mu


def hyperbolicdatagenerator(n:int,mu:float,sigma:float)->list:   
    generated_data = np.random.uniform(0, 1, n)
    dataset = [hyperbolic_inverse(i,mu,sigma) for i in generated_data]
    return dataset


## FONKSİYON PARAMETRELERİ
n=1000
mu=2
sigma=2

x=hyperbolicdatagenerator(n,mu,sigma)

## ÖRNEKLEM PARAMETRELERİ

m1 = np.mean(x)
m2 = np.var(x)
m3 = scipy.stats.skew(x)
m4 = scipy.stats.kurtosis(x)
## ANAKÜTLE PARAMETRELERİ
M1=mu
M2=sigma**sigma



print("Örneklem parametreleri")
print("Ortalama :{:.4f} -Varyans:{:.4f} -çarpıklık: {:.4f} -basıklık :{:.4f}".format(m1,m2,m3,m4))
print("Anakütle  parametreleri")
print("Ortalama :{:.4f} -Varyans:{:.4f} ".format(M1, M2))





Örneklem parametreleri
Ortalama :2.0806 -Varyans:4.1997 -çarpıklık: 0.0947 -basıklık :1.8218
Anakütle  parametreleri
Ortalama :2.0000 -Varyans:4.0000 


### 2.5)Lojistik Dağılım

$$ F(X)=1/2+1/2tanh(1/2(x-a)/b)$$
$$ F(x)^{-1}=2b.arctanh(2(y-1/2))+a $$
$$ f(X)=sech^2((x-a)/2b) / 4b$$
$$ \mu=a$$
$$  \sigma^2=\pi^2b^2/3$$

In [7]:
import numpy as np
import scipy.stats

def logistic_cdf(x:float, a:float, b:float)->float:
    return 0.5 + 0.5 * np.tanh(0.5 * (x - a) / b)

def logistic_pdf(x:float, a:float, b:float)->float:
    sech_term = 1 / np.cosh((x - a) / (2 * b))
    return sech_term**2 / (4 * b)

def logistic_mean(a:float) ->float:
    return a

def logistic_variance(b:float) ->float:
    return (np.pi**2 * b**2) / 3


def inverse_logistic(x:float, a:float, b:float)->float:
    return 2 * b * np.arctanh(2 * (x - 0.5)) + a

def lojisticdatagenerator(n:int, a:float, b:float)->list:   
    generated_data = np.random.uniform(0, 1, n)
    dataset = [inverse_logistic(i, a, b) for i in generated_data]
    return dataset

## FONKSİYON PARAMETRELERİ

n=1000
a = 0.0  # location parameter
b = 1.0  # scale parameter
x=lojisticdatagenerator(n,a,b)

## ÖRNEKLEM PARAMETRELERİ

m1 = np.mean(x)
m2 = np.var(x)
m3 = scipy.stats.skew(x)
m4 = scipy.stats.kurtosis(x)

## ANAKÜTLE PARAMETRELERİ

M1= logistic_mean(a)
M2=logistic_variance(b)

print("Örneklem parametreleri")
print("Ortalama :{:.4f} -Varyans:{:.4f} -çarpıklık: {:.4f} -basıklık :{:.4f}".format(m1,m2,m3,m4))
print("Anakütle  parametreleri")
print("Ortalama :{:.4f} -Varyans:{:.4f} ".format(M1, M2))


Örneklem parametreleri
Ortalama :-0.0001 -Varyans:3.0880 -çarpıklık: 0.0371 -basıklık :0.6548
Anakütle  parametreleri
Ortalama :0.0000 -Varyans:3.2899 


### Sürekli Dağılımların Kaynakçası

Statistical Distributions, Fourth Edition (Catherine Forbes, Merran Evans etc.)(Sürekli dağılımlar)

### 2.6)Benford Dağılımı

kaynakça:https://www.statisticshowto.com/benford-distribution/

$$ f(x)=log_{10}(1+1/x) \quad x=1,2,...,9  $$
$$ F(x)=log_{10}(1+x) \quad x=1,2,...,9 $$
$$F(x)^{-1}=10y-1 $$

In [8]:
import numpy as np
import scipy
def benford_F(x:float)->float:
    return np.log10(1 + x)

def benford_f(x:float)->float:
    return np.log10(1 + 1 / x)

def benford_inverse_F(x:float)->float:
    return 10 *x - 1

def benford_mean()->float:
    expected_value = 0
    for d in range(1, 10):
        probability = np.log10(1 + 1 / d)
        expected_value += d * probability
    return expected_value

def benford_variance()->float:
    expected_value = benford_expected_value()
    variance = 0
    for d in range(1, 10):
        probability = np.log10(1 + 1 / d)
        variance += (d - expected_value) ** 2 * probability
    return variance


def benforddatagenerator(n:int)->list:   
    generated_data = np.random.uniform(0, 1, n)
    dataset = [benford_inverse_F(i) for i in generated_data]
    return dataset



## FONKSİYON PARAMETRELERİ
n=9


x=benforddatagenerator(n)

## ÖRNEKLEM PARAMETRELERİ

m1 = np.mean(x)
m2 = np.var(x)
m3 = scipy.stats.skew(x)
m4 = scipy.stats.kurtosis(x)
## ANAKÜTLE PARAMETRELERİ
M1= benford_expected_value()
M2= benford_variance()



print("Örneklem parametreleri")
print("Ortalama :{:.4f} -Varyans:{:.4f} -çarpıklık: {:.4f} -basıklık :{:.4f}".format(m1,m2,m3,m4))
print("Anakütle  parametreleri")
print("Ortalama :{:.4f} -Varyans:{:.4f} ".format(M1, M2))




NameError: name 'benford_expected_value' is not defined

### 2.7)Geometrik Dağılım


$$ f(k; p) = (1 - p)^{k-1} \cdot p \quad \text{for } k = 1, 2, 3, \ldots $$
$$ F(k; p) = 1 - (1 - p)^k \quad \text{for } k = 1, 2, 3, \ldots$$
$$ F(k; p)^{-1}=ln(1-y)/ln(1-p)  $$
$$ E(x)=1/p $$
$$ Var(x)=1-p/p^2$$
$$ skewness(x)=2-p/\sqrt(1-p)$$
$$ ex kurtosis(x)=6+p^2/(1-p)$$


In [ ]:
import math
import numpy as np
def geometric_pdf(k, p):
    return ((1 - p) ** (k - 1)) * p

def geometric_cdf(k, p):
    return 1 - (1 - p) ** k

def geometric_inverse(k,p):
     return math.log(1 - k) / math.log(1 - p)
    
def geometric_mean(p):
    return 1 / p

def geometric_variance(p):
    return (1 - p) / (p ** 2)

def geometric_skewness(p):
    return (2 - p) / math.sqrt(1 - p)

def geometric_ex_kurtosis(p):
    return 6 + (p ** 2) / (1 - p)

def geometricdatagenerator(n,p):
    generated_data = np.random.uniform(0, 1, n)
    dataset = [geometric_inverse(i,p) for i in generated_data]
    return dataset
    
    
## FONKSİYON PARAMETRELERİ
n=1000
p_value = 0.3
k_value = 5

x=geometricdatagenerator(n,p_value)
## ÖRNEKLEM PARAMETRELERİ

m1 = np.mean(x)
m2 = np.var(x)
m3 = scipy.stats.skew(x)
m4 = scipy.stats.kurtosis(x)
## ANAKÜTLE PARAMETRELERİ
M1 = geometric_mean(p_value)
M2 = geometric_variance(p_value)
M3 = geometric_skewness(p_value)
M4 = geometric_ex_kurtosis(p_value)




print("Örneklem parametreleri")
print("Ortalama :{:.4f} -Varyans:{:.4f} -çarpıklık: {:.4f} -basıklık :{:.4f}".format(m1,m2,m3,m4))
print("Anakütle  parametreleri")
print("Ortalama :{:.4f} -Varyans:{:.4f} -çarpıklık: {:.4f} -basıklık :{:.4f}".format(M1, M2,M3,M4))



### 2.8)Yule–Simon dağılımın

https://www.statisticshowto.com/yule-simon-distribution/


https://en.wikipedia.org/wiki/Yule%E2%80%93Simon_distribution

$$f(x)=\rho \beta(x,\rho+1) $$
$$F(x)=1-x\beta(x,\rho+1) $$
$$F(x)^{-1}=1-y / \beta(x,\rho+1) $$
$$ E(x)=\rho /(\rho -1) \quad p>1 $$
$$ Var(x)=\rho /(\rho -1) \quad p>2 $$
$$ çarpıklık(x)= (\rho+1)^2 \sqrt(p-2) /(\rho-3)\rho \quad \rho>3$$
    $$ex curtosis basıklık(x)=\rho+3+ \frac{11\rho^3-49\rho-22}{(\rho-4)(\rho-3)\rho} \quad \rho >4$$

In [ ]:
import numpy as np
import scipy.special as sps

def yulesimon_pdf(x, rho):
    return rho * sps.beta(x, rho + 1)

def yulesimon_cdf(x, rho):
    return 1 - x * sps.beta(x, rho + 1)

def yulesimon_inverse(y, rho):
    return (1 - y) / sps.beta(y, rho + 1)

def yulesimon_mean(rho):
    if rho > 1:
        return rho / (rho - 1)
    else:
        return np.nan

def yulesimon_variance(rho):
    if rho > 2:
        return rho / ((rho - 1) * (rho - 2))
    else:
        return np.nan

def yulesimon_skewness(rho):
    if rho > 3:
        return ((rho + 1)**2 * np.sqrt(rho - 2)) / ((rho - 3) * rho)
    else:
        return np.nan

def yulesimon_excess_kurtosis(rho):
    if rho > 4:
        return rho + 3 + (11 * rho**3 - 49 * rho - 22) / ((rho - 4) * (rho - 3) * rho)
    else:
        return np.nan
    
    
def yulesimondatagenerator(n,rho):   
    generated_data = np.random.uniform(0, 1, n)
    dataset = [ yulesimon_inverse(i,rho) for i in generated_data]
    return dataset

## FONKSİYON PARAMETRELERİ
rho=5
n=10000

x=yulesimondatagenerator(n,rho)

## ÖRNEKLEM PARAMETRELERİ

m1 = np.mean(x)
m2 = np.var(x)
m3 = scipy.stats.skew(x)
m4 = scipy.stats.kurtosis(x)
## ANAKÜTLE PARAMETRELERİ
M1=yulesimon_mean(rho)
M2=yulesimon_variance(rho)
M3=yulesimon_skewness(rho)
M4=yulesimon_excess_kurtosis(rho)


print("Örneklem parametreleri")
print("Ortalama :{:.4f} -Varyans:{:.4f} -çarpıklık: {:.4f} -basıklık :{:.4f}".format(m1,m2,m3,m4))
print("Anakütle  parametreleri")
print("Ortalama :{:.4f} -Varyans:{:.4f} -çarpıklık: {:.4f} - basıklık :{:.4f}".format(M1, M2,M3,M4))
    
    
    


### 2.9)ZETA dağılımı (BURADA SİMÜLASYON YOK ÖZELLİKLE SORMAK İÇİN EKLEDİM)

https://en.wikipedia.org/wiki/Zeta_distribution


https://stats.libretexts.org/Bookshelves/Probability_Theory/Probability_Mathematical_Statistics_and_Stochastic_Processes_(Siegrist)/05%3A_Special_Distributions/5.40%3A_The_Zeta_Distribution


https://www.randomservices.org/random/special/Zeta.html

zeta fonksiyonu :$$\zeta(a) = \sum_{n=1}^\infty \frac{1}{n^a}, \quad a \in (1, \infty)$$
Harmonic fonksiyonu: $$  H_{k,s} \sum_{n=1}^{k}1/i^s $$

$$ f(n)=1/ \zeta(a)n^a \quad n \epsilon N+$$
$$ F(n)=H_{(k,s)}/ \zeta(a) \quad n \epsilon N+$$
$$ \E\left(N^k\right) = \frac{\zeta(a - k)}{\zeta(a)} $$
$$ \var(N) = \frac{\zeta(a - 2)}{\zeta(a)} - \left(\frac{\zeta(a - 1)}{\zeta(a)}\right)^2 $$
$$ \skw(N) = \frac{\zeta(a - 3) \zeta^2(a) - 3 \zeta(a - 1) \zeta(a - 2) \zeta(a) + 2 \zeta^3(a - 1)}{[\zeta(a - 2) \zeta(a) - \zeta^2(a - 1)]^{3/2}} \quad a>4 $$
$$ \kur(N) = \frac{\zeta(a - 4) \zeta^3(a) - 4 \zeta(a - 1) \zeta(a - 3) \zeta^2(a) + 6 \zeta^2(a - 1) \zeta(a - 2) \zeta(a) - 3 \zeta^4(a - 1)}{\left[\zeta(a - 2) \zeta(a) - \zeta^2(a - 1)\right]^2} \quad  a>5 $$


In [ ]:
import numpy as np
import scipy.special as sps
import math
def zeta(a:float)->float:
    return sps.zeta(a)

def hurwitz(k:int, s:float)->float:
    return np.sum([1 / n**s for n in range(1, k + 1)])

def zeta_pdf(n:int, a:float)->float:
    return 1 / (zeta(a) * n**a)

def zeta_cdf(k:int, s:float, a:float)->float:
    return hurwitz(k, s) / zeta(a)

def zeta_mean(a:float, k:int)->float:
    return zeta(a - k) / zeta(a)

def zeta_variance(a:float)->float:
    return zeta(a - 2) / zeta(a) - (zeta(a - 1) / zeta(a))**2

def zeta_skewness(a:float)->float:
    if a > 4:
        numerator = zeta(a - 3) * zeta(a)**2 - 3 * zeta(a - 1) * zeta(a - 2) * zeta(a) + 2 * zeta(a)**3
        denominator = (zeta(a - 2) * zeta(a) - zeta(a - 1)**2)**(3 / 2)
        return numerator / denominator
    else:
        return np.nan

def zeta_kurtosis(a:float)->float:
    if a > 5:
        numerator = zeta(a - 4) * zeta(a)**3 - 4 * zeta(a - 1) * zeta(a - 3) * zeta(a)**2 + 6 * zeta(a - 1)**2 * zeta(a - 2) * zeta(a) - 3 * zeta(a)**4
        denominator = (zeta(a - 2) * zeta(a) - zeta(a - 1)**2)**2
        return numerator / denominator
    else:
        return np.nan
    
    
    


### 2.10)Pareto dağılımı
https://mathworld.wolfram.com/ParetoDistribution.html
https://www.statisticshowto.com/pareto-distribution/
https://stats.libretexts.org/Bookshelves/Probability_Theory/Probability_Mathematical_Statistics_and_Stochastic_Processes_(Siegrist)/05%3A_Special_Distributions/5.36%3A_The_Pareto_Distribution

$$ f(x)= ab^a/x^{a+1}$$
$$F(x)=1-{(b/x)^a}$$ 
$$F(x)^{-1}=b/(a\sqrt{1-y}) $$
$$ quantile(p)=1/{(1-p^{1/a}))} $$
$$\mu=ab/a-1 $$
$$\sigma^2=ab^2/(a-1)^2(a-2) $$
$$ skew(x)=\sqrt{(a-2)/a }. 2(a+1)/a-3$$
$$ kurt(x)=6(a^3+a^2-6a-2)/a(a-3)(a-4) $$

In [ ]:
import numpy as np
import scipy
def f(x, a, b):
    return a * b**a / x**(a + 1)

def F(x, a, b):
    return 1 - (b / x)**a

def inverse_F(y, a, b):
    return b / math.pow(np.sqrt(1 - y), a)

def quantile(p, a):
    return 1 / (1 - p ** (1 / a))

def mean(a:float, b:float):
    return (a * b) / (a - 1)

def variance(a:float, b:float):
    return (a * (b**2) )/( ((a - 1)**2 * (a - 2)))

def skewness(a:float):
    return np.sqrt((a - 2) / a) * 2 * (a + 1) / (a - 3)

def kurtosis(a:float)->float:
    return 6 * (a**3 + a**2 - 6*a - 2) / (a * (a - 3) * (a - 4))


def paretodatagenerator(n:int,a:float,b:float)->list:   
    generated_data = np.random.uniform(0, 1, n)
    dataset = [ quantile(i,a) for i in generated_data]
    return dataset

## FONKSİYON PARAMETRELERİ

n=9
a=7
b=5
x=paretodatagenerator(n,a,b)

## ÖRNEKLEM PARAMETRELERİ

m1 = np.mean(x)
m2 = np.var(x)
m3 = scipy.stats.skew(x)
m4 = scipy.stats.kurtosis(x)
## ANAKÜTLE PARAMETRELERİ
M1=mean(a, b)
M2=variance(a, b)
M3=skewness(a)
M4=kurtosis(a)


print("Örneklem parametreleri")
print("Ortalama :{:.4f} -Varyans:{:.4f} -çarpıklık: {:.4f} -basıklık :{:.4f}".format(m1,m2,m3,m4))
print("Anakütle  parametreleri")
print("Ortalama :{:.4f} -Varyans:{:.4f} -çarpıklık: {:.4f} - basıklık :{:.4f}".format(M1, M2,M3,M4))

https://www.statisticshowto.com/probability-and-statistics/statistics-definitions/probability-distribution/#az


https://stats.libretexts.org/Bookshelves/Probability_Theory/Probability_Mathematical_Statistics_and_Stochastic_Processes_(Siegrist)/05%3A_Special_Distributions

